In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights, resnet101, ResNet101_Weights
from torch.utils.data import TensorDataset, DataLoader, Dataset
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from sklearn.metrics import f1_score

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 資料處理
先把txt檔整理成 id, 圖檔名, calss 的 DataFrame

In [ ]:

f_id = open('/content/drive/My Drive/DL_HW5/id_filename.txt')
f_label = open('/content/drive/My Drive/DL_HW5/training_class.txt')
id_lines = f_id.readlines()
label_lines = f_label.readlines()

dic_id = {}
dic_label = {}
train_list = []
test_list = []



for line in label_lines:
  name, label = line.split(',')
  name = name.strip()
  label = label.strip()
  train_list.append(name)
  dic_label[name] = label

for line in id_lines:
  ID, name = line.split(',')
  ID = ID.strip()
  name = name.strip()
  dic_id[name] = ID
  if(name in dic_label):
    continue
  else:
    test_list.append(name)

print(dic_id, dic_label)

{'20190720122509800_500.jpg': '0', '20180329230800038_500.jpg': '1', '20170324103832691_500.jpg': '2', '20190321150614058_500.jpg': '3', '20171027094856497_500.jpg': '4', '20170805105749396_500.jpg': '5', '20190324153354132_500.jpg': '6', '20190320143017896_500.jpg': '7', '20190319173139187_500.jpg': '8', '20180729132923908_500.jpg': '9', '20181020113308500_500.jpg': '10', '20180331161801048_500.jpg': '11', '20170324195552604_500.jpg': '12', '20190319143310910_500.jpg': '13', '20180724120643991_500.jpg': '14', '20171028203402794_500.jpg': '15', '20190727082937294_500.jpg': '16', '20170728180115812_500.jpg': '17', '20190720144533556_500.jpg': '18', '20180330033813592_500.jpg': '19', '20180331163441360_500.jpg': '20', '20200105112143260_500.jpg': '21', '20180331190421109_500.jpg': '22', '20180330000736944_500.jpg': '23', '20180723215201130_500.jpg': '24', '20170804114636518_500.jpg': '25', '20180329162805636_500.jpg': '26', '20191102104355938_500.jpg': '27', '20180719175224685_500.jpg': 

In [ ]:
print(len(dic_id),len(dic_label),len(test_list))

1560 1341 219


In [ ]:
img_width, img_height = 224,224

# 設定圖片的轉換
transform = transforms.Compose([
    transforms.RandomRotation(20),  # 隨機旋轉圖像最多20度，減少圖片方向影響
    transforms.Resize((img_width, img_height)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [ ]:
# 讀取圖片並與標籤對應起來
image_list = []
label_list = []


#for file in os.listdir('./image_set')[:count]:
    #if file.endswith('.jpg'):
for name in train_list:
    image_path = os.path.join('/content/drive/My Drive/DL_HW5/photos/', name)
    label = int(dic_label[name])
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image_list.append(image)
    label_list.append(label)

In [ ]:
for i in range(len(label_list)):
  label_list[i] = label_list[i]-1

In [ ]:
label_list_ = torch.Tensor(label_list)
label_list_oh= nn.functional.one_hot(label_list_.to(torch.int64), num_classes=4)

In [ ]:
label_list_oh = label_list_oh.type(torch.FloatTensor)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(image_list, label_list_oh, test_size=0.2, random_state=0)
Trainset = TensorDataset(torch.stack(x_train), y_train)
Validset = TensorDataset(torch.stack(x_valid), y_valid)
trainloader = DataLoader(Trainset, batch_size=32, shuffle=True, num_workers=0)
validloader = DataLoader(Validset, batch_size=1, shuffle=True, num_workers=0)

In [ ]:
def train(trainloader, validloader, model, optimizer, epochs) :

    # loss_MSE = torch.nn.MSELoss(reduction="mean")
    # loss_SSE = torch.nn.MSELoss(reduction="sum")
    criterion = torch.nn.CrossEntropyLoss()

    train_loss_list = []
    output_list = []
    val_acc = 0
    best_acc = 0
    step_cnt = 0
    best_step_cnt = 0


    for epoch in tqdm(range(epochs)):

        # Set current loss value
        losses = []
        running_loss = 0

        # Iterate over data in train loader
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device,dtype=torch.float32)
            labels = labels.to(device,dtype=torch.float32)
            # labels = labels.reshape((-1, 1))
#             print("label shape", labels.shape)
            # labels.type(torch.FloatTensor)
            step_cnt += 1
            model.train()

            optimizer.zero_grad()
            outputs = model(inputs)
            # output_list.append(outputs)
            # outputs = outputs.reshape((-1, 1))
            # outputs.type(torch.FloatTensor)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

             #每 100 個 batch 算一次
            if (step_cnt % 100 == 0):
                train_loss = running_loss / 100
                print('training loss :', train_loss)
                train_loss_list.append(train_loss)

                with torch.no_grad():
                    temp = 0
                    cnt = 0
                    valid_oplist = []
                    valid_truelist = []
                    for j, (vinputs, vlabels) in enumerate(validloader):
                      vinputs = vinputs.to(device,dtype=torch.float32)
                      vlabels = vlabels.to(device,dtype=torch.float32)
                      # vlabels = vlabels.reshape((-1, 1))
                      voutputs = model(vinputs)
                      valid_oplist.append(voutputs)
                      valid_truelist.append(vlabels)

                    acc = 0
                    for k in range(len(valid_oplist)):
                      # print(valid_oplist[k],valid_truelist[k])
                      _, V_labels = torch.max(valid_truelist[k], 1)
                      _, predicted = torch.max(valid_oplist[k], 1)
                      if(predicted == V_labels):
                        acc+=1
                    acc = acc/len(valid_oplist)

                if (acc > best_acc):
                    # step_after_best = 0
                    best_step_cnt = step_cnt
                    best_acc = acc
                    torch.save(model, 'best_model.pt') # Save model parameters
                running_loss = 0 #每100 batch 重置


    return best_step_cnt, best_acc, train_loss_list

In [ ]:
def test(testloader, model):
    # loss_MSE = torch.nn.MSELoss(reduction="mean")
    # test_count = 0
    outputlist = []
    for data in tqdm(testloader):
        x_test = data.to(device)
        op_test = model(x_test)
        _, labels = torch.max(op_test, 1)
        outputlist.append(labels)

    return outputlist

In [ ]:
# import train

num_classes = 4

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

model.to(device)

# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

epochs = 500

### train

best_step_cnt, best_acc, train_loss_list = train(trainloader, validloader, model, optimizer, epochs)

print("best_step_cnt: ",best_step_cnt)
print("best_valid_accuracy: ",best_acc)

  0%|          | 2/500 [00:21<1:30:01, 10.85s/it]

training loss : 0.015886190766468643


  1%|          | 5/500 [00:56<1:32:34, 11.22s/it]

training loss : 0.0019119975611101836


  2%|▏         | 8/500 [01:30<1:31:31, 11.16s/it]

...

 69%|██████▉   | 344/500 [1:06:20<29:04, 11.18s/it]

training loss : 1.3968217172077857e-06


 69%|██████▉   | 347/500 [1:06:55<28:47, 11.29s/it]

training loss : 5.30800152773736e-07


 70%|██████▉   | 348/500 [1:07:08<30:04, 11.87s/it]

In [ ]:
### test
# 讀取圖片並與標籤對應起來
image_test_list = []


#for file in os.listdir('./image_set')[:count]:
    #if file.endswith('.jpg'):
for name in test_list:
    image_path = os.path.join('/content/drive/My Drive/DL_HW5/photos/', name)
    # label = int(dic_label[name])
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image_test_list.append(image)
    # label_test_list.append(label)


testloader = DataLoader(torch.stack(image_test_list), batch_size=1, shuffle=False, num_workers=0)

In [ ]:
model = torch.load('best_model.pt')
model.eval()
test_output = test(testloader, model)

100%|██████████| 219/219 [00:02<00:00, 104.70it/s]


In [ ]:
list_in = list(range(0,len(test_output)))
result = pd.DataFrame(columns =['id','label'], index =list_in)
for i in range(len(test_output)):
  k = test_list[i]
  result['id'][i] = dic_id[k]
  result['label'][i] = int(test_output[i])+1

In [ ]:
result.to_csv('/content/drive/My Drive/DL_HW5/R11725045_0.01.csv', index=False)

In [ ]:
result

,id,label
0,10,3
1,14,2
2,18,1
3,21,3
4,24,2
...,...,...
214,1531,2
215,1541,2
216,1543,2
217,1544,1
